In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import sys
import numpy as np
import random as rn
import pandas as pd


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data understanding and Overview

Read & view dataset :

In [ ]:
import pandas as pd

review_df=pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv").iloc[:10000, :]

review_df.head(10)

show simple describtion of dataset number of positive and negative records:

In [ ]:
print(review_df['sentiment'].value_counts())

ploting bar plot flowshart (comparing number of positive and negative records) :

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(6,5))
sns.set(style = "darkgrid" , font_scale = 1.2)
sns.countplot(review_df.sentiment)

chose just positve reviews of our data frame ,,,to use it in wordcloud visualizing :

In [ ]:
positive_values = review_df[(review_df.review .notnull()) & (review_df.sentiment == "positive")]

positive_values.head(15)

ploting wordcloud for positive values :

In [ ]:
from wordcloud import WordCloud,STOPWORDS 
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
wordcloud = WordCloud(width=500,height=250, max_font_size=80, max_words=150, background_color="white").generate(positive_values.review[0])
f = plt.figure() 
f.set_figwidth(15) 
f.set_figheight(10) 
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()


chose just negative reviews of our data frame ,,,to use it in wordcloud visualizing :

In [ ]:
negative_values=review_df[(review_df.review .notnull()) & (review_df.sentiment == "negative")]

negative_values.head(15)

ploting wordcloud for negative values :

In [ ]:
wordcloud = WordCloud(width=500,height=250, max_font_size=80, max_words=150, background_color="white").generate(negative_values.review[11])


f = plt.figure() 
f.set_figwidth(15) 
f.set_figheight(10) 

 
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.margins(x=0, y=0)
 
plt.show()


# Data preprocessing phase

1. Turn sentiment into categorical values :

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
review_df['sentiment'] = labelencoder.fit_transform(review_df['sentiment'])

review_df['sentiment'].head(10)


2. Remove none text and special character


In [ ]:
%%time

import re
import string
pattern = re.compile(r'<br\s*/><br\s*/>>*|(\-)|(\\)|(\/)')
def preprocess_reviews(reviews):
    reviews = [pattern.sub(" ",item) for item in reviews]
    return reviews
clean = preprocess_reviews(review_df['review'])

review_df['review'] = clean

def remove_punctuation(input):
    table = str.maketrans('','',string.punctuation)
    return input.translate(table)
review_df['review'] = review_df['review'].apply(remove_punctuation)

review_df['review'].head(15)

3. Convert all text to lowercase

In [ ]:
review_df['review'] = review_df['review'].str.lower()

In [ ]:
review_df.head(15)

4. Remove line breaks

In [ ]:
%%time
review_df['review']=review_df['review'].apply(str)
def remove_linebreaks(input):
    text = re.compile(r'\n')
    return text.sub(r' ',input)
review_df['review'] = review_df['review'].apply(remove_linebreaks)


5. Tokenization

In [ ]:
%%time
from nltk.tokenize import word_tokenize
review_df['review'] = review_df['review'].apply(word_tokenize)


In [ ]:
review_df.head(15)

6. Remove stopword

In [ ]:
%%time

from nltk.corpus import stopwords
def remove_stopwords(input1):
    words = []
    for word in input1:
            if word not in stopwords.words('english'):
             words.append(word)
    return words
review_df['review'] = review_df['review'].apply(remove_stopwords)


review_df['review'].head(10)

7. Lemmatization

In [ ]:
%%time
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
def lemma_wordnet(input):
    return [lem.lemmatize(w) for w in input]
review_df['review'] = review_df['review'].apply(lemma_wordnet)

In [ ]:
review_df.head(15)

8. Combine individual words

In [ ]:
%%time
def combine_text(input):
    combined = '  '.join(input)
    return combined
review_df['review'] = review_df['review'].apply(combine_text)

In [ ]:
review_df['review'].head(15)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(10,5))
sns.distplot(review_df['review'].apply(len),kde=False,color='red',hist=True)
plt.xlabel("Rivew Length",size=15)
plt.ylabel("Frequency",size=15)
plt.title("Length Histogram",size=15)

determine X:review , y:seniment (label and target feature of dataset)to use it later in ML models : 

In [ ]:
X=review_df.iloc[:,0:1].values
y=review_df.iloc[:,-1].values


# Word embedding techniques (Text  features extraction) 

**Since we have reviews as text and we want to run a mathematical model we need a method to convert the text to numbers**

A. Bag of words 

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# cv=CountVectorizer(max_features=1000)
# X=cv.fit_transform(review_df['review']).toarray()

B. TF-IDF **

In [ ]:
# import sklearn
# from sklearn.feature_extraction.text import TfidfVectorizer
# tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,2))
# X = tfidf.fit_transform(review_df.review).toarray()

C. Hashing 

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=1000)
X=hv.fit_transform(review_df['review']).toarray()

spliting data into train and test set by using sklearn’s train_test_split :

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Analyzing machine learningModels( classification models )

build and prepare models :

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

#Gaussian Naive Bayes model
clf1=GaussianNB()

#Bernoulli model
clf2=BernoulliNB()

#Ridge Classifier model
clf3=RidgeClassifier()

#Random Forest Classifier model 
clf4 = RFC(n_jobs = 2, random_state = 0)

#Support Vector Machine Classifier model
clf5 = SVC(kernel='linear')

Train models on our dataset :

In [ ]:
clf1.fit(X_train,y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)
clf4.fit(X_train,y_train)
clf5.fit(X_train,y_train)

Evaluating our trined machine learning models :

In [ ]:
y_pred1=clf1.predict(X_test)
y_pred2=clf2.predict(X_test)
y_pred3=clf3.predict(X_test)
y_pred4=clf4.predict(X_test)
y_pred5=clf5.predict(X_test)

acc1=accuracy_score(y_test,y_pred1)
acc2=accuracy_score(y_test,y_pred2)
acc3=accuracy_score(y_test,y_pred3)
acc4=accuracy_score(y_test,y_pred4)
acc5=accuracy_score(y_test,y_pred5)

In [ ]:
#Get acuracy score results for trained models :
print("Gaussian",acc1,"%")
print("Bernaulli",acc2,"%")
print("RidgeClassifier",acc3,"%")
print("Random Forest",acc4,"%")
print("Support Vector Machine",acc5,"%")


**Plotting Models Accuaracy scores :**

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(7,5))
ax = fig.add_axes([0,0,1,1])
models = ['Gaussian ', 'Bernaulli ', 'Ridge ', 'Random Forest ', 'SVM']
accurisy = [acc1*100 ,acc2*100,acc3*100,acc4*100,acc5*100]
ax.bar(models,accurisy,color = 'bgmrc',width = 0.8)
plt.xlabel("ML model",size=15)
plt.ylabel("Accuracy",size=15)
 
plt.show()

Get evaluation metrics of ML models (confusion_matrix ,accuracy , precision,recall,f1-score ,support ) :

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix 

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from yellowbrick.datasets import load_occupancy
from yellowbrick.classifier import classification_report
clas = ['Bad Reviews','Good Reviews']
visualizer = classification_report(
    clf1, X_train, y_train, X_test, y_test, classes=clas, support=True
)

In [ ]:
Co_Mat = confusion_matrix(y_test,y_pred1)
print(Co_Mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test,y_pred1)
plt.figure(figsize = (5,5))
sns.heatmap(cm,cmap= "YlOrRd", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = ['Bad Reviews','Good Reviews'], 
            yticklabels = ['Bad Reviews','Good Reviews'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from yellowbrick.datasets import load_occupancy
from yellowbrick.classifier import classification_report
clas = ['Bad Reviews','Good Reviews']
visualizer = classification_report(
    clf2, X_train, y_train, X_test, y_test, classes=clas, support=True
)

In [ ]:
Co_Mat = confusion_matrix(y_test,y_pred2)
print(Co_Mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test,y_pred2)
plt.figure(figsize = (5,5))
sns.heatmap(cm,cmap= "YlOrRd", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = ['Bad Reviews','Good Reviews'], 
            yticklabels = ['Bad Reviews','Good Reviews'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

As we see RidgeClassifier  model gave the highest accuricy , so lits show it‘s classification report:

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from yellowbrick.datasets import load_occupancy
from yellowbrick.classifier import classification_report
clas = ['Bad Reviews','Good Reviews']
visualizer = classification_report(
    clf3, X_train, y_train, X_test, y_test, classes=clas, support=True
)


In [ ]:
Co_Mat = confusion_matrix(y_test,y_pred3)
print(Co_Mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test,y_pred3)
plt.figure(figsize = (5,5))
sns.heatmap(cm,cmap= "YlOrRd", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = ['Bad Reviews','Good Reviews'], 
            yticklabels = ['Bad Reviews','Good Reviews'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
# print("Rabdom Forest  classification report is :\n")
# print(classification_report(y_test,y_pred4, target_names = ['Bad Reviews','Good Reviews']))
from sklearn.model_selection import TimeSeriesSplit
from yellowbrick.datasets import load_occupancy
from yellowbrick.classifier import classification_report
clas = ['Bad Reviews','Good Reviews']
visualizer = classification_report(
    clf4, X_train, y_train, X_test, y_test, classes=clas, support=True
)

In [ ]:
Co_Mat = confusion_matrix(y_test,y_pred4)
print(Co_Mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test,y_pred4)
plt.figure(figsize = (5,5))
sns.heatmap(cm,cmap= "YlOrRd", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = ['Bad Reviews','Good Reviews'], 
            yticklabels = ['Bad Reviews','Good Reviews'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
# print("SVM  classification report is :\n")
# print(classification_report(y_test,y_pred5, target_names = ['Bad Reviews','Good Reviews']))

from sklearn.model_selection import TimeSeriesSplit
from yellowbrick.datasets import load_occupancy
from yellowbrick.classifier import classification_report
clas = ['Bad Reviews','Good Reviews']
visualizer = classification_report(
    clf5, X_train, y_train, X_test, y_test, classes=clas, support=True
)

In [ ]:
Co_Mat = confusion_matrix(y_test,y_pred5)
print(Co_Mat)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test,y_pred5)
plt.figure(figsize = (5,5))
sns.heatmap(cm,cmap= "YlOrRd", 
            linecolor = 'black', 
            linewidth = 1, 
            annot = True, 
            fmt='', 
            xticklabels = ['Bad Reviews','Good Reviews'], 
            yticklabels = ['Bad Reviews','Good Reviews'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

In [ ]:
pip install pytorch_pretrained_bert

In [ ]:
import torch
from pytorch_pretrained_bert import BertModel
from torch import nn
# from torchnlp.datasets import imdb_dataset      # --> We are using our own uploaded dataset.
from pytorch_pretrained_bert import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
rn.seed(321)
np.random.seed(321)
torch.manual_seed(321)
torch.cuda.manual_seed(321)

In [ ]:
path = '../input/imdb-50k-movie-reviews-test-your-bert/'

# experimenting here with a sample of dataset, to avoid memory overflow.

train_data = pd.read_csv(path + 'train.csv')
test_data = pd.read_csv(path + 'test.csv')




# experimenting here with a sample of dataset, to avoid memory overflow.
train_data = train_data[:7500]
test_data = test_data[:2500]


# Convert the DataFrame to a dictionary , like [{'col1': 1.0, 'col2': 0.5}, {'col1': 2.0, 'col2': 0.75}]
train_data = train_data.to_dict(orient='records')
test_data = test_data.to_dict(orient='records')

In [ ]:
train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['sentiment']), test_data)))

len(train_texts), len(train_labels), len(test_texts), len(test_labels)

visualizing one of the sentences from train set

In [ ]:
train_texts[0]

# STEP 1: Preprocess Data and Build a Transformer Model

In [ ]:
# pytorch_pretained_bert already available in kaggle conda env.
!pip install pytorch-nlp

Preparing Token embeddings

In [ ]:
from pytorch_pretrained_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
tokenizer.tokenize('Here is nashaat jouda welcome ')

In [ ]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:510] + ['[SEP]'], test_texts))

len(train_tokens), len(test_tokens)

Preparing Token Ids..

In [ ]:
train_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, train_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(list(map(tokenizer.convert_tokens_to_ids, test_tokens)), maxlen=512, truncating="post", padding="post", dtype="int")

train_tokens_ids.shape, test_tokens_ids.shape

In [ ]:
train_y = np.array(train_labels) == 'pos'
test_y = np.array(test_labels) == 'pos'
train_y.shape, test_y.shape, np.mean(train_y), np.mean(test_y)

In [ ]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks =  [[float(i > 0) for i in ii] for ii in test_tokens_ids]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [ ]:
# baseline_model = make_pipeline(CountVectorizer(ngram_range=(1,3)), LogisticRegression()).fit(train_texts, train_labels)

In [ ]:
# baseline_predicted = baseline_model.predict(test_texts)

In [ ]:
# print(classification_report(test_labels, baseline_predicted))

# BERT Model

In [ ]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [ ]:
bert_clf = BertBinaryClassifier()
bert_clf = bert_clf.cuda()     # running BERT on CUDA_GPU

In [ ]:
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [ ]:
x = torch.tensor(train_tokens_ids[:3]).to(device)
y, pooled = bert_clf.bert(x, output_all_encoded_layers=False)
x.shape, y.shape, pooled.shape

In [ ]:
y = bert_clf(x)
y.cpu().detach().numpy()        # kinda Garbage Collector to free up used and cache space

In [ ]:
# Cross- checking CUDA GPU Memory to ensure GPU memory is not overflowing.
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [ ]:
y, x, pooled = None, None, None
torch.cuda.empty_cache()     # Clearing Cache space for fresh Model run
str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

Fine Tune BERT..

In [ ]:
# Setting hyper-parameters

BATCH_SIZE = 4
EPOCHS = 3

In [ ]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()

test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

str(torch.cuda.memory_allocated(device)/1000000 ) + 'M'

In [ ]:
train_dataset = TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
param_optimizer = list(bert_clf.sigmoid.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

In [ ]:
optimizer = Adam(bert_clf.parameters(), lr=3e-6)

In [ ]:
torch.cuda.empty_cache()   # Clearing Cache space for a fresh Model run

In [ ]:
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)
        print(str(torch.cuda.memory_allocated(device)/1000000 ) + 'M')
        logits = bert_clf(token_ids, masks)
        
        loss_func = nn.BCELoss()

        batch_loss = loss_func(logits, labels)
        train_loss += batch_loss.item()
        
        
        bert_clf.zero_grad()
        batch_loss.backward()
        

        clip_grad_norm_(parameters=bert_clf.parameters(), max_norm=1.0)
        optimizer.step()
        
        clear_output(wait=True)
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])

In [ ]:
np.mean(bert_predicted)

In [ ]:
print(classification_report(test_y, bert_predicted))